In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#import warnings
#warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
import matplotlib.pyplot as plt

import plotly.express as px 
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import datetime
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from collections import Counter
import random
import string
import datetime as dt 
from datetime import datetime

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [2]:
#pip install feature-engine
#pip install mlxtend

SyntaxError: invalid syntax (<ipython-input-2-e5f1c61f8f7b>, line 1)

In [16]:
rfm_melt = pd.read_csv("rfm_melt_random.csv")
rfm_melt = rfm_melt.drop(columns=["Metric","Value"])
rfm_melt.head()
rfm_melt.info()

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,customer_id,K_Cluster
0,0002YJ,3
1,000986,3
2,000CMS,3
3,000H95,1
4,000IS0,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2329554 entries, 0 to 2329553
Data columns (total 2 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   customer_id  object
 1   K_Cluster    int64 
dtypes: int64(1), object(1)
memory usage: 35.5+ MB


In [17]:
# drop duplicates because each customer_id appears 3 times row-wise; respresented recency, frequency, monetary for clustering. 
rfm_melt = rfm_melt.drop_duplicates(subset=["customer_id"], keep="first")
rfm_melt.head()
rfm_melt.info()

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,customer_id,K_Cluster
0,0002YJ,3
1,000986,3
2,000CMS,3
3,000H95,1
4,000IS0,2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 776518 entries, 0 to 776517
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_id  776518 non-null  object
 1   K_Cluster    776518 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 17.8+ MB


In [18]:
data_full = pd.read_csv("cleaned_data.csv")
data_full = data_full.iloc[: , 1:]
data_full.head()

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,order_date,customer_id,order_id,product_group,detailed_category,product_id,product_name,sku,qty_ordered,gross_revenue,net_revenue
0,2022-04-10,F125YX,7269150,Emma Others,Beds,12164,Emma Revor BSB 180x200 - BG Bedbase P2,EBDRV180200C32,1,282.85,212.66
1,2022-04-29,RY8A7X,7426895,Emma Mattress Protector V3,Accessories,7458,Emma Mattress Protector V3 200x200,ESSES200200AAC,1,96.75,68.13
2,2022-05-22,ZPUQOH,7588817,Emma Others,Accessories,8293,Emma Foam Pillow 80x40 V2,EPWFP080040AAB,1,54.40,41.01
3,2022-01-02,TSK3PM,6401748,Emma ONE Mattress medium,Basic Mattress,3138,Emma One Medium 090x200,EMAON090200CAC,2,362.02,304.21
4,2022-04-10,OC9N75,7262778,Emma Others,Basic Mattress,4570,Emma Dynamic Mattress-180 X 200 cm,EMADY180200AAA,1,441.75,332.12


In [19]:
data = pd.merge(data_full, rfm_melt, on="customer_id", how="left")
data.head()

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,order_date,customer_id,order_id,product_group,detailed_category,product_id,product_name,sku,qty_ordered,gross_revenue,net_revenue,K_Cluster
0,2022-04-10,F125YX,7269150,Emma Others,Beds,12164,Emma Revor BSB 180x200 - BG Bedbase P2,EBDRV180200C32,1,282.85,212.66,5
1,2022-04-29,RY8A7X,7426895,Emma Mattress Protector V3,Accessories,7458,Emma Mattress Protector V3 200x200,ESSES200200AAC,1,96.75,68.13,4
2,2022-05-22,ZPUQOH,7588817,Emma Others,Accessories,8293,Emma Foam Pillow 80x40 V2,EPWFP080040AAB,1,54.40,41.01,4
3,2022-01-02,TSK3PM,6401748,Emma ONE Mattress medium,Basic Mattress,3138,Emma One Medium 090x200,EMAON090200CAC,2,362.02,304.21,0
4,2022-04-10,OC9N75,7262778,Emma Others,Basic Mattress,4570,Emma Dynamic Mattress-180 X 200 cm,EMADY180200AAA,1,441.75,332.12,0


In [20]:
data_mba = pd.DataFrame(data[["order_id","product_name","qty_ordered","K_Cluster"]])

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
cust_T0 = data_mba[data_mba["K_Cluster"]==0]
cust_T1 = data_mba[data_mba["K_Cluster"]==1]
cust_T2 = data_mba[data_mba["K_Cluster"]==2]
cust_T3 = data_mba[data_mba["K_Cluster"]==3]
cust_T4 = data_mba[data_mba["K_Cluster"]==4]
cust_T5 = data_mba[data_mba["K_Cluster"]==5]
cust_T6 = data_mba[data_mba["K_Cluster"]==6]

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Customer Segment Overview

In [52]:
rfm_kmeans = pd.read_csv("rfm_kmeans.csv")
rfm_kmeans = pd.merge(rfm_kmeans, rfm_melt, on="customer_id", how="left")
rfm_kmeans.head()

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,customer_id,recency,frequency,monetary,K_Cluster
0,0002YJ,395,1,298.32,3
1,000986,483,1,155.22,3
2,000CMS,536,1,166.32,3
3,000H95,628,1,57.60,1
4,000IS0,723,2,358.20,2


In [53]:
overview_T0 = rfm_kmeans[rfm_kmeans["K_Cluster"]==0]
overview_T1 = rfm_kmeans[rfm_kmeans["K_Cluster"]==1]
overview_T2 = rfm_kmeans[rfm_kmeans["K_Cluster"]==2]
overview_T3 = rfm_kmeans[rfm_kmeans["K_Cluster"]==3]
overview_T4 = rfm_kmeans[rfm_kmeans["K_Cluster"]==4]
overview_T5 = rfm_kmeans[rfm_kmeans["K_Cluster"]==5]
overview_T6 = rfm_kmeans[rfm_kmeans["K_Cluster"]==6]

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [54]:
overview_T0.describe()
overview_T1.describe()
overview_T2.describe()
overview_T3.describe()
overview_T4.describe()
overview_T5.describe()
overview_T6.describe()

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,recency,frequency,monetary,K_Cluster
count,118350.000000,118350.000000,118350.000000,118350.0
mean,186.912041,1.662459,509.718585,0.0
std,103.562300,0.605190,127.816044,0.0
min,1.000000,1.000000,124.800000,0.0
25%,107.000000,1.000000,421.600000,0.0
50%,186.000000,2.000000,484.050000,0.0
75%,261.000000,2.000000,580.300000,0.0
max,503.000000,4.000000,1328.100000,0.0


,recency,frequency,monetary,K_Cluster
count,156211.000000,156211.000000,156211.000000,156211.0
mean,727.275486,1.035747,270.050025,1.0
std,98.768717,0.185658,111.915092,0.0
min,573.000000,1.000000,14.400000,1.0
25%,631.000000,1.000000,179.100000,1.0
50%,711.000000,1.000000,280.200000,1.0
75%,810.000000,1.000000,358.200000,1.0
max,943.000000,2.000000,754.600000,1.0


,recency,frequency,monetary,K_Cluster
count,84194.000000,84194.000000,84194.000000,84194.0
mean,625.144226,2.266777,461.533338,2.0
std,135.768731,0.533875,134.968023,0.0
min,256.000000,1.000000,88.590000,2.0
25%,505.000000,2.000000,358.200000,2.0
50%,614.000000,2.000000,441.960000,2.0
75%,720.000000,2.000000,542.400000,2.0
max,943.000000,6.000000,1432.800000,2.0


,recency,frequency,monetary,K_Cluster
count,182224.000000,182224.000000,182224.000000,182224.0
mean,436.946247,1.062687,259.472388,3.0
std,79.885149,0.242399,103.437080,0.0
min,270.000000,1.000000,7.160000,3.0
25%,364.000000,1.000000,179.100000,3.0
50%,444.000000,1.000000,271.200000,3.0
75%,498.000000,1.000000,350.240000,3.0
max,591.000000,2.000000,626.340000,3.0


,recency,frequency,monetary,K_Cluster
count,41994.000000,41994.000000,41994.000000,41994.0
mean,253.868648,3.658332,809.995303,4.0
std,180.427016,0.893193,218.081495,0.0
min,1.000000,1.000000,156.170000,4.0
25%,113.000000,3.000000,653.650000,4.0
50%,221.000000,4.000000,781.525000,4.0
75%,349.000000,4.000000,940.830000,4.0
max,942.000000,8.000000,2392.000000,4.0


,recency,frequency,monetary,K_Cluster
count,13033.000000,13033.000000,13033.000000,13033.0
mean,180.612215,7.187984,1509.292153,5.0
std,153.313927,1.763561,629.942166,0.0
min,1.000000,1.000000,463.220000,5.0
25%,75.000000,7.000000,1195.280000,5.0
50%,135.000000,7.000000,1480.540000,5.0
75%,252.000000,8.000000,1702.390000,5.0
max,943.000000,91.000000,29417.700000,5.0


,recency,frequency,monetary,K_Cluster
count,180512.000000,180512.000000,180512.000000,180512.0
mean,150.782740,1.099611,207.134474,6.0
std,82.682508,0.313949,108.450479,0.0
min,1.000000,1.000000,2.200000,6.0
25%,93.000000,1.000000,108.800000,6.0
50%,146.000000,1.000000,208.050000,6.0
75%,216.000000,1.000000,299.000000,6.0
max,322.000000,4.000000,419.400000,6.0


### Market Basket Analysis

In [24]:
def encoder(x):
    if x <= 0:
        return 0
    else:
        return 1

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### T0

In [25]:
market_basket_T0 = cust_T0.groupby(["order_id","product_name"])["qty_ordered"].sum().unstack().reset_index().fillna(0).set_index('order_id')
market_basket_T0 = market_basket_T0.applymap(encoder)
T0_items = apriori(market_basket_T0, min_support = 0.01, use_colnames = True)
T0_items
T0_rules = association_rules(T0_items, metric = 'lift', min_threshold = 1)
T0_rules

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.027623,(Emma Dynamic Mattress-100 X 200 cm)
1,0.042891,(Emma Dynamic Mattress-140 X 200 cm)
2,0.036506,(Emma Dynamic Mattress-160 X 200 cm)
3,0.061129,(Emma Dynamic Mattress-180 X 200 cm)
4,0.012047,(Emma Dynamic Mattress-80 X 200 cm)
5,0.091651,(Emma Dynamic Mattress-90 X 200 cm)
6,0.109281,(Emma Foam Pillow 80x40 V2)
7,0.014609,(Emma Mattress Protector V3 090x200)
8,0.020873,(Emma Mattress Protector V3 140x200)
9,0.011396,(Emma Mattress Protector V3 160x200)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Emma One Medium 090x200),(Emma One 090x200 hard),0.090169,0.066204,0.023278,0.258157,3.899397,0.017308,1.258751
1,(Emma One 090x200 hard),(Emma One Medium 090x200),0.066204,0.090169,0.023278,0.351606,3.899397,0.017308,1.403206


In [292]:
T0 = T0_rules
T0 = T0.iloc[::2].sort_values("lift",ascending=False).reset_index(drop=True)
T0.drop(["leverage",'antecedent support','consequent support'], axis=1,inplace=True)
pd.set_option("display.max_colwidth",100)
T0.to_csv("T0_rules.csv", index=False)

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### T1

In [26]:
market_basket_T1 = cust_T1.groupby(["order_id","product_name"])["qty_ordered"].sum().unstack().reset_index().fillna(0).set_index('order_id')
market_basket_T1 = market_basket_T1.applymap(encoder)
T1_items = apriori(market_basket_T1, min_support = 0.01, use_colnames = True)
T1_items
T1_rules = association_rules(T1_items, metric = 'lift', min_threshold = 1)
T1_rules

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.012811,(Emma Microfiber Pillow 080x040)
1,0.011008,(Emma Microfiber Pillow 080x080)
2,0.017658,(Emma One 080x200 hard)
3,0.152402,(Emma One 090x200 hard)
4,0.047006,(Emma One 100 x 200 medium)
5,0.035750,(Emma One 100x200 hard)
6,0.067340,(Emma One 140 x 200 hard)
7,0.187973,(Emma One Medium 090x200)
8,0.025201,(Emma One V3 Mattress 080x200 Medium)
9,0.014581,(Emma One V3 Mattress 120x200 Medium)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


### T2

In [82]:
market_basket_T2 = cust_T2.groupby(["order_id","product_name"])["qty_ordered"].sum().unstack().reset_index().fillna(0).set_index('order_id')
market_basket_T2 = market_basket_T2.applymap(encoder)
T2_items = apriori(market_basket_T2, min_support = 0.01, use_colnames = True)
#T2_items
T2_rules = association_rules(T2_items, metric = 'lift', min_threshold = 1)
T2_rules

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Emma New Slatted Frame - 140x200 cm),(Emma One V3 Mattress 140x200 Medium),0.030511,0.101389,0.019059,0.624636,6.160793,0.015965,2.393974
1,(Emma One V3 Mattress 140x200 Medium),(Emma New Slatted Frame - 140x200 cm),0.101389,0.030511,0.019059,0.187974,6.160793,0.015965,1.193914
2,(Emma One 090x200 hard),(Emma New Slatted Frame - 90x200 cm),0.235154,0.047291,0.014945,0.063554,1.343887,0.003824,1.017367
3,(Emma New Slatted Frame - 90x200 cm),(Emma One 090x200 hard),0.047291,0.235154,0.014945,0.316020,1.343887,0.003824,1.118229
4,(Emma One Medium 090x200),(Emma New Slatted Frame - 90x200 cm),0.287792,0.047291,0.023616,0.082059,1.735194,0.010006,1.037876
5,(Emma New Slatted Frame - 90x200 cm),(Emma One Medium 090x200),0.047291,0.287792,0.023616,0.499374,1.735194,0.010006,1.422636
6,(Emma One V3 Mattress 080x200 Medium),(Emma One 080x200 hard),0.047508,0.036184,0.021909,0.461171,12.745321,0.020190,1.788724
7,(Emma One 080x200 hard),(Emma One V3 Mattress 080x200 Medium),0.036184,0.047508,0.021909,0.605507,12.745321,0.020190,2.414471
8,(Emma One 090x200 hard),(Emma One Medium 090x200),0.235154,0.287792,0.130934,0.556800,1.934734,0.063258,1.606969
9,(Emma One Medium 090x200),(Emma One 090x200 hard),0.287792,0.235154,0.130934,0.454960,1.934734,0.063258,1.403284


In [293]:
T2 = T2_rules
T2["antecedents"] = pd.DataFrame(list(T2_rules.antecedents), columns=['antecedents'])
T2["consequents"] = pd.DataFrame(list(T2_rules.consequents), columns=['consequents'])
T2 = T2.iloc[::2].sort_values("lift",ascending=False).reset_index(drop=True)
T2.drop(["leverage",'antecedent support','consequent support'], axis=1,inplace=True)
pd.set_option("display.max_colwidth",100)
T2.to_csv("T2_rules.csv", index=False)

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### T3

In [13]:
market_basket_T3 = cust_T3.groupby(["order_id","product_name"])["qty_ordered"].sum().unstack().reset_index().fillna(0).set_index('order_id')
market_basket_T3 = market_basket_T3.applymap(encoder)
T3_items = apriori(market_basket_T3, min_support = 0.01, use_colnames = True)
T3_items
T3_rules = association_rules(T3_items, metric = 'lift', min_threshold = 1)
T3_rules

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.013982,(Emma Dynamic Mattress-90 X 200 cm)
1,0.028673,(Emma Foam Pillow 80x40 V2)
2,0.015277,(Emma Microfiber Pillow 080x040 – Overfilling)
3,0.012811,(Emma Microfiber Pillow 080x080 – Overfilling)
4,0.012551,(Emma One 080x200 hard)
5,0.115052,(Emma One 090x200 hard)
6,0.048568,(Emma One 100 x 200 medium)
7,0.028923,(Emma One 100x200 hard)
8,0.054773,(Emma One 140 x 200 hard)
9,0.202833,(Emma One Medium 090x200)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


### T4

In [301]:
market_basket_T4 = cust_T4.groupby(["order_id","product_name"])["qty_ordered"].sum().unstack().reset_index().fillna(0).set_index('order_id')
market_basket_T4 = market_basket_T4.applymap(encoder)
T4_items = apriori(market_basket_T4, min_support = 0.01, use_colnames = True)
T4_items.head()
T4_rules = association_rules(T4_items, metric = 'lift', min_threshold = 1)
T4_rules

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.012944,(Emma Box Bed 140x200 - DG Bed Base Part 1 V3)
1,0.016842,(Emma Box Bed 140x200 - F1 Bed Base Part 2 V3)
2,0.010995,(Emma Box Bed 160x200 - DG Bed Base Part 1 V3)
3,0.014655,(Emma Box Bed 160x200 - F1 Bed Base Part 2 V3)
4,0.019549,(Emma Box Bed 180x200 - DG Bed Base Part 1 V3)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Emma Box Bed 140x200 - DG Bed Base Part 1 V3),(Emma Box Bed 140x200 - F1 Bed Base Part 2 V3),0.012944,0.016842,0.012051,0.931034,55.281406,0.011833,14.255795
1,(Emma Box Bed 140x200 - F1 Bed Base Part 2 V3),(Emma Box Bed 140x200 - DG Bed Base Part 1 V3),0.016842,0.012944,0.012051,0.715548,55.281406,0.011833,3.470024
2,(Emma Box Bed 180x200 - DG Tufted HBoard V3),(Emma Box Bed 180x200 - DG Bed Base Part 1 V3),0.012869,0.019549,0.012869,1.000000,51.152207,0.012618,inf
3,(Emma Box Bed 180x200 - DG Bed Base Part 1 V3),(Emma Box Bed 180x200 - DG Tufted HBoard V3),0.019549,0.012869,0.012869,0.658295,51.152207,0.012618,2.888841
4,(Emma Box Bed 180x200 - F1 Bed Base Part 2 V3),(Emma Box Bed 180x200 - DG Bed Base Part 1 V3),0.024742,0.019549,0.017362,0.701744,35.895746,0.016879,3.287277
5,(Emma Box Bed 180x200 - DG Bed Base Part 1 V3),(Emma Box Bed 180x200 - F1 Bed Base Part 2 V3),0.019549,0.024742,0.017362,0.888128,35.895746,0.016879,8.717613
6,(Emma Box Bed 180x200 - DG Tufted HBoard V3),(Emma Box Bed 180x200 - F1 Bed Base Part 2 V3),0.012869,0.024742,0.011367,0.883237,35.698071,0.011048,8.352458
7,(Emma Box Bed 180x200 - F1 Bed Base Part 2 V3),(Emma Box Bed 180x200 - DG Tufted HBoard V3),0.024742,0.012869,0.011367,0.459411,35.698071,0.011048,1.826027
8,(Emma Dynamic Mattress-90 X 200 cm),(Emma Foam Pillow 80x40 V2),0.048874,0.150445,0.011010,0.225266,1.497335,0.003657,1.096577
9,(Emma Foam Pillow 80x40 V2),(Emma Dynamic Mattress-90 X 200 cm),0.150445,0.048874,0.011010,0.073180,1.497335,0.003657,1.026226


In [303]:
T4 = T4_rules
T4 = T4.iloc[::2].sort_values("lift",ascending=False).reset_index(drop=True)
T4.drop(["leverage",'antecedent support','consequent support'], axis=1,inplace=True)
pd.set_option("display.max_colwidth",100)
T4.to_csv("T4_rules.csv", index=False)

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### T5

In [117]:
market_basket_T5 = cust_T5.groupby(["order_id","product_name"])["qty_ordered"].sum().unstack().reset_index().fillna(0).set_index('order_id')
market_basket_T5 = market_basket_T5.applymap(encoder)
T5_items = apriori(market_basket_T5, min_support = 0.05, use_colnames = True)
T5_items
T5_rules = association_rules(T5_items, metric = 'lift', min_threshold = 1)
T5_rules

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.056840,(Emma BSB Entry - 180x200 BedBase P1)
1,0.056840,(Emma BSB Entry - 180x200 Plain Hboard)
2,0.115728,(Emma BSB Entry - Black Feet)
3,0.075856,(Emma Boxspring Bed - Black Oak Feet)
4,0.098655,(Emma Boxspring Bed - Natural Oak Feet)
5,0.070078,(Emma Dynamic Mattress-180 X 200 cm)
6,0.133484,(Emma Foam Pillow 80x40 V2)
7,0.070288,(Emma Helix Mattress BSB hybrid 180x200)
8,0.051534,(Emma Helix Mattress BSB hybrid 200x200)
9,0.052952,(Emma One Medium 090x200)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Emma BSB Entry - 180x200 Plain Hboard),(Emma BSB Entry - 180x200 BedBase P1),0.056840,0.056840,0.056840,1.000000,17.593346,0.053609,inf
1,(Emma BSB Entry - 180x200 BedBase P1),(Emma BSB Entry - 180x200 Plain Hboard),0.056840,0.056840,0.056840,1.000000,17.593346,0.053609,inf
2,(Emma Dynamic Mattress-180 X 200 cm),(Emma BSB Entry - 180x200 BedBase P1),0.070078,0.056840,0.056787,0.810345,14.256677,0.052804,4.973027
3,(Emma BSB Entry - 180x200 BedBase P1),(Emma Dynamic Mattress-180 X 200 cm),0.056840,0.070078,0.056787,0.999076,14.256677,0.052804,1006.175877
4,(Emma Topper 180x200),(Emma BSB Entry - 180x200 BedBase P1),0.219952,0.056840,0.051271,0.233102,4.101052,0.038769,1.229839
5,(Emma BSB Entry - 180x200 BedBase P1),(Emma Topper 180x200),0.056840,0.219952,0.051271,0.902033,4.101052,0.038769,7.962380
6,(Emma Dynamic Mattress-180 X 200 cm),(Emma BSB Entry - 180x200 Plain Hboard),0.070078,0.056840,0.056787,0.810345,14.256677,0.052804,4.973027
7,(Emma BSB Entry - 180x200 Plain Hboard),(Emma Dynamic Mattress-180 X 200 cm),0.056840,0.070078,0.056787,0.999076,14.256677,0.052804,1006.175877
8,(Emma Topper 180x200),(Emma BSB Entry - 180x200 Plain Hboard),0.219952,0.056840,0.051271,0.233102,4.101052,0.038769,1.229839
9,(Emma BSB Entry - 180x200 Plain Hboard),(Emma Topper 180x200),0.056840,0.219952,0.051271,0.902033,4.101052,0.038769,7.962380


In [295]:
T5 = T5_rules
T5 = T5.iloc[::2].sort_values("lift",ascending=False).reset_index(drop=True)
T5.drop(["leverage",'antecedent support','consequent support'], axis=1,inplace=True)
pd.set_option("display.max_colwidth",100)
#T5 = T5[T5["support"] >= 0.05]
T5.to_csv("T0_rules.csv", index=False)

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### T6

In [16]:
market_basket_T6 = cust_T6.groupby(["order_id","product_name"])["qty_ordered"].sum().unstack().reset_index().fillna(0).set_index('order_id')
market_basket_T6 = market_basket_T6.applymap(encoder)
T6_items = apriori(market_basket_T6, min_support = 0.01, use_colnames = True)
T6_items
T6_rules = association_rules(T6_items, metric = 'lift', min_threshold = 1)
T6_rules

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.015688,(Emma Diamond Degree Topper 180x200)
1,0.014411,(Emma Dynamic Mattress-100 X 200 cm)
2,0.042890,(Emma Dynamic Mattress-140 X 200 cm)
3,0.044398,(Emma Dynamic Mattress-90 X 200 cm)
4,0.172340,(Emma Foam Pillow 80x40 V2)
5,0.011224,(Emma Microfiber Duvet 135x200)
6,0.066991,(Emma Microfiber Pillow 080x040 – Overfilling)
7,0.063890,(Emma Microfiber Pillow 080x080 – Overfilling)
8,0.046727,(Emma One 090x200 hard)
9,0.010505,(Emma One 100 x 200 medium)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


### Aggregate Market Basket Analysis

In [285]:
agg1_rules = pd.read_csv("agg1_rules.csv")
agg1_rules["antecedents"] = agg1_rules["antecedents"].str.split("{\'",n=1,expand=True)[1].str.split("\'}",n=1,expand=True)[0].str.replace("\'","")
agg1_rules["consequents"] = agg1_rules["consequents"].str.split("{\'",n=1,expand=True)[1].str.split("\'}",n=1,expand=True)[0].str.replace("\'","")

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [286]:
agg1 = agg1_rules
agg1 = agg1.iloc[::2].sort_values("lift",ascending=False).reset_index(drop=True)
agg1.drop(["leverage",'antecedent support','consequent support'], axis=1,inplace=True)
pd.set_option("display.max_colwidth",100)
#agg1[agg1['support'] >= 0.002]
#agg1.to_csv("agg1_result.csv")

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [287]:
agg2_rules = pd.read_csv("agg2_rules.csv")
agg2_rules["antecedents"] = agg2_rules["antecedents"].str.split("{\'",n=1,expand=True)[1].str.split("\'}",n=1,expand=True)[0].str.replace("\'","")
agg2_rules["consequents"] = agg2_rules["consequents"].str.split("{\'",n=1,expand=True)[1].str.split("\'}",n=1,expand=True)[0].str.replace("\'","")

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [296]:
agg2 = agg2_rules
agg2 = agg2.iloc[::2].sort_values("lift",ascending=False).reset_index(drop=True)
agg2.drop(["leverage",'antecedent support','consequent support'], axis=1,inplace=True)
pd.set_option("display.max_colwidth",100)
#T5 = T5[T5["support"] >= 0.05]
agg2.to_csv("agg2_rules_new.csv", index=False)

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [290]:
agg3_rules = pd.read_csv("agg3_rules.csv")
agg3_rules["antecedents"] = agg3_rules["antecedents"].str.split("{\'",n=1,expand=True)[1].str.split("\'}",n=1,expand=True)[0].str.replace("\'","")
agg3_rules["consequents"] = agg3_rules["consequents"].str.split("{\'",n=1,expand=True)[1].str.split("\'}",n=1,expand=True)[0].str.replace("\'","")

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [297]:
agg3 = agg3_rules
agg3 = agg3.iloc[::2].sort_values("lift",ascending=False).reset_index(drop=True)
agg3.drop(["leverage",'antecedent support','consequent support'], axis=1,inplace=True)
pd.set_option("display.max_colwidth",100)
#T5 = T5[T5["support"] >= 0.05]
agg3.to_csv("agg3_rules_new.csv", index=False)

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Aggregate Market Basket Analysis

In [ ]:
cust_agg = data_mba.copy().drop(columns=["K_Cluster"])
market_basket_agg = cust_agg.groupby(["order_id","product_name"])["qty_ordered"].sum().unstack().reset_index().fillna(0).set_index('order_id')
market_basket_agg = market_basket_agg.applymap(encoder)

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
agg1_items = apriori(market_basket_agg, min_support = 0.001, use_colnames = True)
agg1_items
agg1_rules = association_rules(agg1_items, metric = 'lift', min_threshold = 1)
agg1_rules

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [ ]:
agg2_items = apriori(market_basket_agg, min_support = 0.005, use_colnames = True)
agg2_items
agg2_rules = association_rules(agg2_items, metric = 'lift', min_threshold = 1)
agg2_rules

In [ ]:
agg3_items = apriori(market_basket_agg, min_support = 0.01, use_colnames = True)
agg3_items
agg3_rules = association_rules(agg3_items, metric = 'lift', min_threshold = 1)
agg3_rules